In [1]:
#If th openai api key is not set already, please set it here.
import os
if 'OPENAI_API_KEY' not in os.environ:  
     os.environ['OPENAI_API_KEY'] = 'your key'

In [2]:
from amadeusgpt import AMADEUS
from amadeusgpt.config import Config
from amadeusgpt.utils import parse_result
import amadeusgpt
from amadeusgpt import create_project
import matplotlib.pyplot as plt
import cv2

### Note that unlike other notebooks, we don't have keypoint_file_path here (as it's not provided)
### By default, we use gpt-4o to determine which SuperAnimal models to run and it will run SuperAnimal in the first time the keypoints related queries are asked
### Make sure you use a short video clips if you are not using GPUs in Linux (Mac silicon support to be added)

In [3]:
scene_frame_number = 400

# where you store you video and (optionally) keypoint files
data_folder = "temp_data_folder"
result_folder = "temp_result_folder"
video_suffix = ".mp4"

config = create_project(data_folder, result_folder, video_suffix = video_suffix)

config["scene_frame_number"] = scene_frame_number

amadeus = AMADEUS(config)
video_file_paths = amadeus.get_video_file_paths()
print (video_file_paths)

Project created at temp_result_folder. Results will be saved to temp_result_folder
The project will load video files (*..mp4) and optionally keypoint files from temp_data_folder
A copy of the project config file is saved at temp_result_folder/config.yaml
{'data_info': {'data_folder': 'temp_data_folder',
               'result_folder': 'temp_result_folder',
               'video_suffix': '.mp4'},
 'llm_info': {'max_tokens': 4096, 'temperature': 1.0}}
No video files found in the data folder temp_data_folder. Please check the data folder and the video suffix


AttributeError: 'AMADEUS' object has no attribute 'sandbox'

In [ ]:
query = "Plot the trajectory of the animal using the animal center and color it by time"
qa_message = amadeus.step(query)
parse_result(amadeus, qa_message)

In [ ]:
query = "give me the occupancy plot using the animal center"
qa_message = amadeus.step(query)
qa_message = parse_result(amadeus, qa_message)